Для начала импортируем все необходимые нам библиотеки и данные

Устанавливаем библиотеку surprise, если у вас её нет:

In [ ]:
!pip install surprise

In [ ]:
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd

In [ ]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

Наши данные содержат информацию о фильмах и их жанрах:

In [ ]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


А также информацию о том, какие рейтинги поставили пользователи фильмам:

In [ ]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


Для удобства объединим данные из двух табличек в одну и почистим недостающие данные (пропуски в потенциальной таблице user-item):

In [ ]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [ ]:
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09


Вот так мы можем по текущей таблице посмотреть, например, какие фильмы смотрел пользователь с id=2.0:

In [ ]:
movies_with_ratings[movies_with_ratings.userId == 2.0].title.unique()

array(['Shawshank Redemption, The (1994)', 'Tommy Boy (1995)',
       'Good Will Hunting (1997)', 'Gladiator (2000)',
       'Kill Bill: Vol. 1 (2003)', 'Collateral (2004)',
       'Talladega Nights: The Ballad of Ricky Bobby (2006)',
       'Departed, The (2006)', 'Dark Knight, The (2008)',
       'Step Brothers (2008)', 'Inglourious Basterds (2009)',
       'Zombieland (2009)', 'Shutter Island (2010)',
       'Exit Through the Gift Shop (2010)', 'Inception (2010)',
       'Town, The (2010)', 'Inside Job (2010)',
       'Louis C.K.: Hilarious (2010)', 'Warrior (2011)',
       'Dark Knight Rises, The (2012)',
       'Girl with the Dragon Tattoo, The (2011)',
       'Django Unchained (2012)', 'Wolf of Wall Street, The (2013)',
       'Interstellar (2014)', 'Whiplash (2014)', 'The Drop (2014)',
       'Ex Machina (2015)', 'Mad Max: Fury Road (2015)',
       'The Jinx: The Life and Deaths of Robert Durst (2015)'],
      dtype=object)

Классы в библиотеке surprise умеют принимать формат данных только определенного типа, поэтому для этого создаем новую таблицу с переименованными колонками:

In [ ]:
dataset = movies_with_ratings[["title", "userId", "rating"]].rename({
    "userId": "uid",
    "title": "iid"
})

In [ ]:
dataset.head()

,title,userId,rating
0,Toy Story (1995),1.0,4.0
1,Toy Story (1995),5.0,4.0
2,Toy Story (1995),7.0,4.5
3,Toy Story (1995),15.0,2.5
4,Toy Story (1995),17.0,4.5


Классы в surprise принимают данные через свой отдельный класс Dataset, в который мы отправляем данные и класс Reader с указанным диапазоном допустимых оценок:

In [ ]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

Также качество нашего алгоритма мы захотим потом проверить, поэтому предлагаю поделить данные на тренировочные и тестовые, чтобы потом замерить качество на отложенной выборке:

In [ ]:
trainset, testset = train_test_split(data, test_size=0.15)

Реализация классической коллаборативной фильтрации находится в классе KNNWithMeans, будем использовать её:

In [ ]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


Делаем предсказание на тестовом датасете:

In [ ]:
test_pred = algo.test(testset)

Считаем RMSE:

In [ ]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8749


0.8749123394712246

А вот так теперь можно предсказывать оценку пользователя для фильма:

In [ ]:
algo.predict(uid=2, iid='Fight Club (1999)').est

3.5010617197526543